<a href="https://colab.research.google.com/github/Xessen/Mask_Detection/blob/main/Covid_Mask_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2,os,random
import numpy as np

def ImagePreprocess(path2img,train_size=0.7,size=(80,80)):
    x_test=[]
    y_test=[[]]
    x_train=[]
    y_train=[[]]
    for path in path2img:
        numberoffiles=len(os.listdir(path))
        counter=0
        for img in os.listdir(path):
            if counter<(numberoffiles*train_size):
                pic = cv2.cvtColor(cv2.imread(os.path.join(path,img)),cv2.COLOR_BGR2RGB)
                pic = cv2.resize(pic,size)
                pic =pic.reshape((size[0],size[1],3))
                x_train.append([pic])
                y_train[0].append(path2img.index(path))
                counter+=1
            else:
                pic = cv2.cvtColor(cv2.imread(os.path.join(path,img)),cv2.COLOR_BGR2RGB)
                pic = cv2.resize(pic,size)
                pic =pic.reshape((size[0],size[1],3))
                x_test.append([pic])
                y_test[0].append(path2img.index(path))
                counter+=1

    temp0=list(zip(x_train,y_train[0]))
    random.shuffle(temp0)
    x_train,y_train[0]=zip(*temp0)
    temp1=list(zip(x_test,y_test[0]))
    random.shuffle(temp1)
    x_test,y_test[0]=zip(*temp1)

    x_train=np.array(x_train)
    y_train=np.array(y_train)
    x_test=np.array(x_test)
    y_test=np.array(y_test)

    x_train=x_train/255.
    x_test=x_test/255.

    return x_train,x_test,y_train,y_test

In [3]:
from google.colab import files
#Uploading kaggle.json file that has API Key
files.upload()
!pip install -q kaggle

Saving kaggle.json to kaggle.json


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset
!chmod 600 /root/.kaggle/kaggle.json

 97% 319M/330M [00:03<00:00, 62.4MB/s]
100% 330M/330M [00:03<00:00, 92.5MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip

In [7]:
path=["/content/Face Mask Dataset/Train/WithoutMask","/content/Face Mask Dataset/Train/WithMask"]
x_train,x_test,y_train,y_test=ImagePreprocess(path)

In [8]:
from keras.utils import to_categorical
encoded_train=to_categorical(y_train)
encoded_test=to_categorical(y_test)

In [9]:
from tensorflow.keras import models
from tensorflow.keras.layers import Dense,Flatten,Input,Conv2D,MaxPooling2D,Dropout
import tensorflow as tf

In [10]:
x_train=np.reshape(x_train,(7000,80,80,3))
x_test=np.reshape(x_test,(3000,80,80,3))

In [20]:
cnn=models.Sequential(
    [
     Conv2D(filters=64,kernel_size=(5,5),activation="relu",input_shape=(80,80,3)),
     Dropout(0.2),
     MaxPooling2D((3,3)),
     Conv2D(filters=32,kernel_size=(5,5),activation="relu"),
     Dropout(0.2),
     MaxPooling2D((3,3)),
     Conv2D(filters=32,kernel_size=(5,5),activation="relu"),
     Dropout(0.2),
     MaxPooling2D((3,3)),
     Flatten(),
     Dense(64,"relu"),
     Dense(2,"sigmoid")
    ]
)
cnn.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [21]:
cnn.fit(x_train,encoded_train[0],epochs=5,batch_size=128)

Epoch 1/5
55/55 [==============================] - 97s 2s/step - loss: 0.4683 - accuracy: 0.7913
Epoch 2/5
55/55 [==============================] - 97s 2s/step - loss: 0.1659 - accuracy: 0.9376
Epoch 3/5
55/55 [==============================] - 97s 2s/step - loss: 0.1100 - accuracy: 0.9616
Epoch 4/5
55/55 [==============================] - 97s 2s/step - loss: 0.0803 - accuracy: 0.9677
Epoch 5/5
55/55 [==============================] - 97s 2s/step - loss: 0.0406 - accuracy: 0.9855


In [22]:
accuracy,loss=cnn.evaluate(x_test,encoded_test[0],batch_size=128)

24/24 [==============================] - 8s 340ms/step - loss: 0.0744 - accuracy: 0.9867
